In [0]:
WITH StagedEvents AS (
    SELECT
        CampaignId,
        ContactId,
        EventType,
        -- Assign a stage number based on the EventType
        CASE
            WHEN EventType = 'sent' THEN 1
            WHEN EventType = 'delivered' THEN 2
            WHEN EventType IN ('spam', 'html_open') THEN 3
            WHEN EventType IN ('click', 'optout_click') THEN 4
        END AS Stage,
        -- Use ROW_NUMBER to pick the first event if a contact has multiple events in the same stage category
        ROW_NUMBER() OVER(PARTITION BY CampaignId, ContactId,
            CASE
                WHEN EventType = 'sent' THEN 1
                WHEN EventType = 'delivered' THEN 2
                WHEN EventType IN ('spam', 'html_open') THEN 3
                WHEN EventType IN ('click', 'optout_click') THEN 4
            END
        ORDER BY EventDate ASC) as rn
    FROM
        silver_events -- <<< IMPORTANT: Replace this with the actual name of your table
    WHERE
        -- The query is filterable by CampaignId. Change the value below as needed.
        CampaignId like "%%"
),

-- CTE 2: ContactJourney
-- This step pivots the staged event data. It transforms the rows of events for each contact
-- into a single row with columns for each stage, representing their complete journey.
ContactJourney AS (
    SELECT
        CampaignId,
        ContactId,
        MAX(CASE WHEN Stage = 1 THEN EventType END) AS stage_1,
        MAX(CASE WHEN Stage = 2 THEN EventType END) AS stage_2,
        MAX(CASE WHEN Stage = 3 THEN EventType END) AS stage_3,
        MAX(CASE WHEN Stage = 4 THEN EventType END) AS stage_4
    FROM
        StagedEvents
    WHERE
        rn = 1 -- This ensures we only use the first event for each stage per contact
    GROUP BY
        CampaignId,
        ContactId
)

-- Final SELECT Statement
-- This aggregates the unique journeys from the ContactJourney CTE and counts
-- the number of unique contacts for each journey.
SELECT
    stage_1,
    stage_2,
    stage_3,
    stage_4,
    COUNT(ContactId) AS value
FROM
    ContactJourney
GROUP BY
    stage_1,
    stage_2,
    stage_3,
    stage_4
ORDER BY
    value DESC;

Databricks visualization. Run in Databricks to view.